## Stop-signal dataset

This is based on the famous stop-signal dataset of Adam Aron and was fetched from openfmri.

See [ds008](https://openfmri.org/dataset/ds000008)

Below we fit a model using the [stopsignal](https://github.com/twiecki/stopsignal/tree/master/stopsignal) Python-package, which underlies Dora Matzke's BEESTs-package.

It's an easy way to fit a censoring race model of two exgausses. One corresponds to the Go-process. The other one corresponds to the Stop process. The mean of the Stop-process ExGauss is a nice estimate of the stop signal RT.

In [ ]:
import stopsignal

In [ ]:
import re

In [ ]:
reg = re.compile('.*/sub-(?P<subj_idx>[0-9]+)/func/sub-[0-9]+_task-(?P<task>[a-z]+)_run-(?P<run>[0-9]+)_events.tsv')

In [ ]:
import glob
import os

In [ ]:
fns = glob.glob(os.path.join(os.environ['HOME'], 'data/openfmri/ds008/sub*/func/sub-*_task-stopsignal_run-*_events.tsv'))

In [ ]:
import pandas

In [ ]:
dfs = []

for fn in fns:
    d = reg.match(fn).groupdict()
    
    dfs.append(pandas.read_csv(fn, sep='\t', index_col=False,na_values=['n/a', 'junk']))
    dfs[-1]['subj_idx'] = d['subj_idx']
    dfs[-1]['run'] = d['run']
    
df = pandas.concat(dfs)

In [ ]:
df = df[~df.response_time.isnull() & (~df.trial_type.isnull())]

In [ ]:
df[(df.subj_idx == 14) & (df.trial_type == '')]

In [ ]:
df['rt'] = df['response_time'] * 1000.
df['ssd'] = (df['SSD']*1000)
df.loc[(df.trial_type == 'go'), 'ssd'] = -999
df['ssd'] = df.ssd.astype(int)
df['inhibited'] = df.trial_type.map({'go':-999, 'failed stop':0, 'successful stop':1})
df['ss_presented'] = df.trial_type.map({'go':0, 'failed stop':1, 'successful stop':1}).astype(int)

In [ ]:
df.trial_type.unique()

In [ ]:
df = df[~((df.subj_idx == '14') & (df.rt > 500))]

In [ ]:
import seaborn as sns
%matplotlib inline

In [ ]:
sns.distplot(df[(df.subj_idx == '14')].rt)

In [ ]:
df[(df.subj_idx == '14')].groupby(['trial_type', 'inhibited', 'ss_presented']).size()

In [ ]:
df.groupby(['subj_idx', 'trial_type']).ssd.min()

In [ ]:
df[(df.subj_idx == '14')][['rt', 'ssd', 'inhibited', 'ss_presented']].groupby(['ss_presented', 'inhibited']).mean()

In [ ]:
model = stopsignal.StopSignal(df)

In [ ]:
model.sample(10000)

In [ ]:
model.plot_posteriors()

In [ ]:
subj_nodes = model.get_subj_nodes()
subj_nodes = subj_nodes[subj_nodes.knode_name == 'mu_stop_subj']
tmp = subj_nodes[['subj_idx', 'tag', 'node', 'mean']]

In [ ]:
tmp.set_index('subj_idx')

In [ ]:
tmp.to_csv('ssrts.csv')

In [ ]:
subj_nodes = model.get_subj_nodes()
subj_nodes = subj_nodes[subj_nodes.knode_name == 'mu_go_subj']
tmp = subj_nodes[['subj_idx', 'tag', 'node', 'mean']]

In [ ]:
tmp.to_csv('mu_go.csv')